In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np


In [5]:
articles_path = "/content/drive/My Drive/nyt_articles_sentiment_scaled.csv"
articledf = pd.read_csv(articles_path)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',10000)
print(articledf.head())

                                               title                                            snippet                                                url            published_date       section subsection sentiment_label  sentiment_score  cluster  scaling_score  scaled_score
0  Federal Reserve Won’t Backpedal on Interest Ra...  Ms Yellen the Fed chairwoman told senators tha...  https://www.nytimes.com/2016/02/12/business/ec...  2016-02-11T19:09:41+0000  Business Day    Economy        positive         0.648025        0            1.3      0.842433
1  The Cities on the Sunny Side of the American E...  In places like Denver Seattle and Atlanta empl...  https://www.nytimes.com/2016/04/01/business/ec...  2016-03-31T18:54:38+0000  Business Day    Economy        positive         0.403958        0            1.3      0.525146
2               City by City, Measuring the Recovery  Cities like Denver San Jose Calif and Austin T...  https://www.nytimes.com/interactive/2016/04/01...  2016-03-31T16

In [20]:
startDate = "2016-01-01"
endDate = "2024-11-23"

articledf['published_date'] = pd.to_datetime(articledf['published_date'])
articledf['week'] = articledf['published_date'].dt.to_period('W').apply(lambda r: r.start_time)
weekly_sentiment = articledf.groupby('week')['scaled_score'].mean().reset_index()


weekly_sentiment['Start Date'] = weekly_sentiment['week']
weekly_sentiment['End Date'] = weekly_sentiment['week'] + pd.Timedelta(days=6)

weekly_sentiment = weekly_sentiment.rename(columns={
    'scaled_score': 'Sentiment Score'
})

weeklydf = weekly_sentiment[['Start Date', 'End Date', 'Sentiment Score']]
print(weeklydf)


<ipython-input-20-8ae867d9fdda>:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  articledf['week'] = articledf['published_date'].dt.to_period('W').apply(lambda r: r.start_time)


    Start Date   End Date  Sentiment Score
0   2016-01-04 2016-01-10         0.505057
1   2016-01-11 2016-01-17         0.443757
2   2016-01-18 2016-01-24         0.556918
3   2016-01-25 2016-01-31         0.425082
4   2016-02-01 2016-02-07         0.505660
..         ...        ...              ...
341 2024-10-21 2024-10-27         0.528765
342 2024-10-28 2024-11-03         0.561272
343 2024-11-04 2024-11-10         0.513694
344 2024-11-11 2024-11-17         0.432878
345 2024-11-18 2024-11-24         0.428538

[346 rows x 3 columns]


In [21]:
lambda_decay = 0.9
for lag in range(1, 4):  # Lag 1 to Lag 3
    weeklydf[f'sentiment_weighted_lag_{lag}'] = weeklydf['Sentiment Score'].shift(lag) * (lambda_decay ** lag)

# Forward fill missing values in the lagged features
weeklydf.bfill(inplace=True)


print(weeklydf)

save_path = "/content/drive/My Drive/weekly_sentiment_lag.csv"
weeklydf.to_csv(save_path, index=False)

    Start Date   End Date  Sentiment Score  sentiment_weighted_lag_1  sentiment_weighted_lag_2  sentiment_weighted_lag_3
0   2016-01-04 2016-01-10         0.505057                  0.454551                  0.409096                  0.368186
1   2016-01-11 2016-01-17         0.443757                  0.454551                  0.409096                  0.368186
2   2016-01-18 2016-01-24         0.556918                  0.399381                  0.409096                  0.368186
3   2016-01-25 2016-01-31         0.425082                  0.501226                  0.359443                  0.368186
4   2016-02-01 2016-02-07         0.505660                  0.382574                  0.451103                  0.323499
..         ...        ...              ...                       ...                       ...                       ...
341 2024-10-21 2024-10-27         0.528765                  0.470155                  0.481904                  0.328837
342 2024-10-28 2024-11-03       